O objetivo desse projeto é realizar uma Análise Exploratória de Dados para responder as seguintes perguntas sobre o Dataset:
* Existe alguma relação entre o total gasto e o método de pagamento utilizado?
* Há um padrão entre os produtos que são consumidos na loja e os que são levados para serem consumidos fora do estabelecimento?
* Quais itens são comprados em maior quantidade? Qual a relação disso com seus respectivos preços?

## Importando as bibliotecas utilizadas

In [ ]:
import pandas as pd

## Lendo o arquivo

In [2]:
base_cafe = pd.read_csv(r'C:\Users\dantt\Documents\Códigos\Data Science e ML - Udemy\CafeSales_EDA\data\raw\dirty_cafe_sales.csv')

## Inspeção inicial dos dados

In [3]:
base_cafe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Transaction ID    10000 non-null  object
 1   Item              9667 non-null   object
 2   Quantity          9862 non-null   object
 3   Price Per Unit    9821 non-null   object
 4   Total Spent       9827 non-null   object
 5   Payment Method    7421 non-null   object
 6   Location          6735 non-null   object
 7   Transaction Date  9841 non-null   object
dtypes: object(8)
memory usage: 625.1+ KB


In [4]:
base_cafe.describe()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
count,10000,9667,9862,9821,9827,7421,6735,9841
unique,10000,10,7,8,19,5,4,367
top,TXN_1961373,Juice,5,3.0,6.0,Digital Wallet,Takeaway,UNKNOWN
freq,1,1171,2013,2429,979,2291,3022,159


In [5]:
base_cafe.head(10)

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11
5,TXN_2602893,Smoothie,5,4.0,20.0,Credit Card,NaN,2023-03-31
6,TXN_4433211,UNKNOWN,3,3.0,9.0,ERROR,Takeaway,2023-10-06
7,TXN_6699534,Sandwich,4,4.0,16.0,Cash,UNKNOWN,2023-10-28
8,TXN_4717867,NaN,5,3.0,15.0,NaN,Takeaway,2023-07-28
9,TXN_2064365,Sandwich,5,4.0,20.0,NaN,In-store,2023-12-31


Analisando as informações, é possível perceber dois pontos críticos:
* Valores faltantes: há uma quantidade significativa de valores ausentes ou indeterminados que precisarão de tratamento;
* Tipagem incorreta: todas as colunas são do tipo 'Object'. Isso é um obstáculo para análises estatísticas e requer tratamento para converter os dados para os tipos corretos.

## Tratamento dos dados

### Valores ausentes

Durante esta etapa, a ideia é preencher os valores nulos das colunas não numéricas com 'UNKNOWN'. Já no caso de colunas numéricas, vamos converter os tipos para, posteriormente, preenchermos com valores consistentes

Visualizando valores nulos/indefinidos

In [6]:
base_cafe.isnull().sum()

Transaction ID         0
Item                 333
Quantity             138
Price Per Unit       179
Total Spent          173
Payment Method      2579
Location            3265
Transaction Date     159
dtype: int64

#### Coluna 'Item'

In [7]:
base_cafe['Item'].isnull().sum()

np.int64(333)

In [8]:
base_cafe['Item'].value_counts()

Item
Juice       1171
Coffee      1165
Salad       1148
Cake        1139
Sandwich    1131
Smoothie    1096
Cookie      1092
Tea         1089
UNKNOWN      344
ERROR        292
Name: count, dtype: int64

Vamos passar os valores nulos da coluna 'Item' para 'UNKNOWN'

In [9]:
# Preenchendo os valores NaN
base_cafe['Item'] = base_cafe['Item'].fillna('UNKNOWN')


In [10]:
# Verificando
base_cafe[base_cafe['Item'].isnull()]

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date


In [11]:
# Substituindo 'ERROR' por 'UNKNOWN'
base_cafe['Item'] = base_cafe['Item'].replace('ERROR', 'UNKNOWN')

# Verificando
base_cafe[base_cafe['Item'] == 'ERROR']

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date


In [12]:
base_cafe['Item'].value_counts()

Item
Juice       1171
Coffee      1165
Salad       1148
Cake        1139
Sandwich    1131
Smoothie    1096
Cookie      1092
Tea         1089
UNKNOWN      969
Name: count, dtype: int64

#### Coluna 'Payment Method'

In [13]:
base_cafe['Payment Method'].isnull().sum() # Checando a quantidade de valores nulos em 'Payment Method'

np.int64(2579)

In [14]:
base_cafe['Payment Method'].value_counts()

Payment Method
Digital Wallet    2291
Credit Card       2273
Cash              2258
ERROR              306
UNKNOWN            293
Name: count, dtype: int64

Vamos passar os valores nulos da coluna 'Payment Method' para 'UNKNOWN'

In [15]:
# Preenchendo os valores NaN
base_cafe['Payment Method'] = base_cafe['Payment Method'].fillna('UNKNOWN')

In [16]:
# Verificando
base_cafe[base_cafe['Payment Method'].isnull()]

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date


In [17]:
# Substituindo 'ERROR' por 'UNKNOWN'
base_cafe['Payment Method'] = base_cafe['Payment Method'].replace('ERROR', 'UNKNOWN')

# Verificando
base_cafe['Payment Method'].value_counts()

Payment Method
UNKNOWN           3178
Digital Wallet    2291
Credit Card       2273
Cash              2258
Name: count, dtype: int64

#### Coluna 'Location'

In [18]:
base_cafe['Location'].isnull().sum() # Checando a quantidade de valores nulos em 'Location'

np.int64(3265)

In [19]:
base_cafe['Location'].value_counts()

Location
Takeaway    3022
In-store    3017
ERROR        358
UNKNOWN      338
Name: count, dtype: int64

Vamos passar os valores nulos da coluna 'Location' para 'UNKNOWN'

In [20]:
# Preenchendo os valores NaN
base_cafe['Location'] = base_cafe['Location'].fillna('UNKNOWN')

In [21]:
# Verificando
base_cafe['Location'].isnull().sum()

np.int64(0)

In [22]:
# Substituindo 'ERROR' por 'UNKNOWN'
base_cafe['Location'] = base_cafe['Location'].replace('ERROR', 'UNKNOWN')

# Verificando
base_cafe['Location'].value_counts()

Location
UNKNOWN     3961
Takeaway    3022
In-store    3017
Name: count, dtype: int64

### Colunas numéricas

#### Coluna 'Quantity'

In [23]:
# Convertendo o tipo
base_cafe['Quantity'] = pd.to_numeric(base_cafe['Quantity'], errors='coerce')

In [24]:
base_cafe['Quantity'].value_counts()

Quantity
5.0    2013
2.0    1974
4.0    1863
3.0    1849
1.0    1822
Name: count, dtype: int64

In [25]:
base_cafe['Quantity'].isnull().sum()

np.int64(479)

#### Coluna 'Price Per Unit'

In [26]:
# Convertendo o tipo
base_cafe['Price Per Unit'] = pd.to_numeric(base_cafe['Price Per Unit'], errors='coerce')

In [27]:
base_cafe['Price Per Unit'].value_counts()

Price Per Unit
3.0    2429
4.0    2331
2.0    1227
5.0    1204
1.0    1143
1.5    1133
Name: count, dtype: int64

In [28]:
base_cafe['Price Per Unit'].isnull().sum()

np.int64(533)

#### Coluna 'Total Spent'

In [29]:
# Convertendo o tipo
base_cafe['Total Spent'] = pd.to_numeric(base_cafe['Total Spent'], errors='coerce')

In [30]:
# Verificando
base_cafe['Total Spent'].value_counts()

Total Spent
6.0     979
12.0    939
3.0     930
4.0     923
20.0    746
15.0    734
8.0     677
10.0    524
2.0     497
9.0     479
5.0     468
16.0    444
25.0    259
7.5     237
1.0     232
4.5     225
1.5     205
Name: count, dtype: int64

In [31]:
base_cafe['Total Spent'].isnull().sum()

np.int64(502)

#### Coluna 'Transaction Date' (Datetime)

In [32]:
# Convertendo o tipo
base_cafe['Transaction Date'] = pd.to_datetime(base_cafe['Transaction Date'], errors='coerce', dayfirst= False)

In [33]:
# Verificando
base_cafe['Transaction Date'].value_counts()

Transaction Date
2023-06-16    40
2023-02-06    40
2023-09-21    39
2023-03-13    39
2023-07-24    39
              ..
2023-11-24    15
2023-04-27    15
2023-07-22    14
2023-03-11    14
2023-02-17    14
Name: count, Length: 365, dtype: int64

In [34]:
base_cafe['Transaction Date'].isnull().sum()

np.int64(460)

### Preenchendo as colunas numéricas com valores consistentes

Preços: vamos utilizar o preço que já conhecemos de cada item para preencher o que está faltando. Por exemplo: se sabemos, por outra transação, que um cookie custa 1.0, podemos substituir o NaN de alguma outra transação envolvendo cookie, já que sabemos o preço dele.

In [35]:
coluna_auxiliar = base_cafe.groupby('Item')['Price Per Unit'].transform('median') # coluna auxiliar que agrupa os preços de cada item
base_cafe['Price Per Unit'] = base_cafe['Price Per Unit'].fillna(coluna_auxiliar) # preenchendo os preços vazios com os preços já conhecidos

In [36]:
# Verificando
base_cafe[base_cafe['Price Per Unit'].isnull()]

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date


Agora, vamos preencher as colunas 'Quantity' e 'Total Spent'

Caso 1: quando temos o preço e o total gasto, mas não temos a quantidade

In [37]:
filtro_quantity = base_cafe['Quantity'].isnull() & base_cafe['Total Spent'].notnull()
base_cafe.loc[filtro_quantity, 'Quantity'] = base_cafe.loc[filtro_quantity, 'Total Spent'] / base_cafe.loc[filtro_quantity, 'Price Per Unit']

Caso 2: quando temos o preço e a quantidade, mas não temos o total gasto

In [38]:
filtro_total = base_cafe['Total Spent'].isnull() & base_cafe['Quantity'].notnull()
base_cafe.loc[filtro_total, 'Total Spent'] = base_cafe.loc[filtro_total, 'Quantity'] * base_cafe.loc[filtro_total, 'Price Per Unit']

In [39]:
base_cafe['Quantity'].isnull().sum()

np.int64(20)

In [40]:
base_cafe['Total Spent'].isnull().sum()

np.int64(20)

Ainda sobraram alguns valores nulos em 'Quantity' e 'Total Spent'. Porém, esses valores representam uma porcentagem pequena do DataSet e podem ser preenchidos sem alterar significativamente os resultados das análises.

No caso de 'Quantity', preenchemos com 1, pois é uma quantidade padrão de compra para diversos produtos

In [41]:
base_cafe['Quantity'] = base_cafe['Quantity'].fillna(1.0)

Para 'Total Spent', podemos fazer a multiplicação 'Quantity' x 'Price Per Unit', pois já preenchemos a 'Quantity' com 1.0

In [42]:
# Total Spent = Quantity x Price Per Unit
base_cafe['Total Spent'] = base_cafe['Total Spent'].fillna(base_cafe['Quantity'] * base_cafe['Price Per Unit'])

Agora, vamos converter a coluna 'Quantity', devidamente preenchida, para o tipo inteiro

In [43]:
base_cafe['Quantity'] = base_cafe['Quantity'].astype(int)

#### Correção na Coluna 'Item'

Alguns produtos tem preços exclusivos, ou seja, é possível identificar o produto pelo seu preço, por exemplo: o Cookie é o único produto que custa 1.0. Sabendo disso, podemos substituir parte dos valores 'UNKNOWN' da coluna 'Item' e obter ainda mais informações consistentes do Dataset a partir desse preenchimento.

In [44]:
base_cafe[base_cafe['Item'] == 'UNKNOWN']

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
6,TXN_4433211,UNKNOWN,3,3.0,9.0,UNKNOWN,Takeaway,2023-10-06
8,TXN_4717867,UNKNOWN,5,3.0,15.0,UNKNOWN,Takeaway,2023-07-28
14,TXN_8915701,UNKNOWN,2,1.5,3.0,UNKNOWN,In-store,2023-03-21
30,TXN_1736287,UNKNOWN,5,2.0,10.0,Digital Wallet,UNKNOWN,2023-06-02
31,TXN_8927252,UNKNOWN,2,1.0,2.0,Credit Card,UNKNOWN,2023-11-06
...,...,...,...,...,...,...,...,...
9951,TXN_4122925,UNKNOWN,4,1.0,4.0,UNKNOWN,Takeaway,2023-10-20
9958,TXN_4125474,UNKNOWN,2,5.0,10.0,Credit Card,In-store,2023-08-02
9981,TXN_4583012,UNKNOWN,5,4.0,20.0,Digital Wallet,UNKNOWN,2023-02-27
9994,TXN_7851634,UNKNOWN,4,4.0,16.0,UNKNOWN,UNKNOWN,2023-01-08


In [45]:
# Filtros para facilitar a substituição || Apenas os itens que tem preços exclusivos
filtro_cookie = (base_cafe['Item'] == 'UNKNOWN') & (base_cafe['Price Per Unit'] == 1.0)
filtro_tea = (base_cafe['Item'] == 'UNKNOWN') & (base_cafe['Price Per Unit'] == 1.5)
filtro_coffee = (base_cafe['Item'] == 'UNKNOWN') & (base_cafe['Price Per Unit'] == 2.0)
filtro_salad = (base_cafe['Item'] == 'UNKNOWN') & (base_cafe['Price Per Unit'] == 5.0)

In [46]:
# Se custa 1.0 e não sei o nome, é Cookie.
base_cafe.loc[filtro_cookie, 'Item'] = 'Cookie'

# Se custa 1.5 e não sei o nome, é Tea.
base_cafe.loc[filtro_tea, 'Item'] = 'Tea'

# Se custa 2.0 e não sei o nome, é Coffee.
base_cafe.loc[filtro_coffee, 'Item'] = 'Coffee'

# Se custa 5.0 e não sei o nome, é Salad.
base_cafe.loc[filtro_salad, 'Item'] = 'Salad'

In [47]:
base_cafe[base_cafe['Item'] == 'UNKNOWN']

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
6,TXN_4433211,UNKNOWN,3,3.0,9.0,UNKNOWN,Takeaway,2023-10-06
8,TXN_4717867,UNKNOWN,5,3.0,15.0,UNKNOWN,Takeaway,2023-07-28
36,TXN_6855453,UNKNOWN,4,3.0,12.0,UNKNOWN,In-store,2023-07-17
61,TXN_8051289,UNKNOWN,1,3.0,3.0,UNKNOWN,In-store,2023-10-09
69,TXN_8471743,UNKNOWN,5,3.0,15.0,Digital Wallet,In-store,2023-04-06
...,...,...,...,...,...,...,...,...
9918,TXN_2292088,UNKNOWN,1,4.0,4.0,Digital Wallet,Takeaway,2023-03-04
9946,TXN_8807600,UNKNOWN,1,4.0,4.0,Cash,Takeaway,2023-09-24
9981,TXN_4583012,UNKNOWN,5,4.0,20.0,Digital Wallet,UNKNOWN,2023-02-27
9994,TXN_7851634,UNKNOWN,4,4.0,16.0,UNKNOWN,UNKNOWN,2023-01-08


Dessa forma, diminuimos em mais de 400 o número de itens que antes eram desconhecidos, mas puderam ser identificados a partir dos seus preços

---

#### Salvando o dataset final limpo:

Com um Dataset muito mais consistente, podemos salvá-lo e realizar análises com base nas informações presentes nele

In [53]:
base_cafe.to_csv(r'C:\Users\dantt\Documents\Códigos\Data Science e ML - Udemy\CafeSales_EDA\data\processed\clean_cafe_sales.csv', index=False)

---